# Data Import

## Introduction

This notebook shows how to import data into a DuckDB database. It uses the `duckdb` Python package to connect to a DuckDB database and import data from various formats, including CSV, JSON, DataFrame, parquet, GeoJSON, Shapefile, GeoParquet, and more.

## Datasets

The following datasets are used in this notebook. You don't need to download them, they can be accessed directly from the notebook.

- [cities.csv](https://open.gishub.org/data/duckdb/cities.csv)
- [countries.csv](https://open.gishub.org/data/duckdb/countries.csv)

## Installation

Uncomment the following cell to install the required packages if needed.

In [ ]:
# %pip install duckdb leafmap

## Library Import

In [7]:
import duckdb
import leafmap
import pandas as pd

## Installing Extensions

DuckDB’s Python API provides functions for installing and loading extensions, which perform the equivalent operations to running the `INSTALL` and `LOAD` SQL commands, respectively. An example that installs and loads the [httpfs extension](https://duckdb.org/docs/extensions/httpfs) looks like follows:

In [8]:
con = duckdb.connect("spatial.db")

In [9]:
con.install_extension("httpfs")
con.load_extension("httpfs")

In [10]:
con.install_extension("spatial")
con.load_extension("spatial")

In [11]:
con.sql('SELECT * FROM duckdb_extensions();')

┌──────────────────┬─────────┬───────────┬──────────────────────┬──────────────────────────────────┬───────────────────┐
│  extension_name  │ loaded  │ installed │     install_path     │           description            │      aliases      │
│     varchar      │ boolean │  boolean  │       varchar        │             varchar              │     varchar[]     │
├──────────────────┼─────────┼───────────┼──────────────────────┼──────────────────────────────────┼───────────────────┤
│ arrow            │ false   │ false     │                      │ A zero-copy data integration b…  │ []                │
│ autocomplete     │ false   │ false     │                      │ Adds support for autocomplete …  │ []                │
│ aws              │ false   │ false     │                      │ Provides features that depend …  │ []                │
│ azure            │ false   │ false     │                      │ Adds a filesystem abstraction …  │ []                │
│ excel            │ false   │ f

## Downloading Sample Data

In [12]:
url = "https://open.gishub.org/data/duckdb/cities.zip"
leafmap.download_file(url, unzip=True)

cities.zip already exists. Skip downloading. Set overwrite=True to overwrite.


'd:\\DataScience\\AIAg\\book\\duckdb\\cities.zip'

## CSV Files

CSV files can be read using the `read_csv` function, called either from within Python or directly from within SQL. By default, the `read_csv` function attempts to auto-detect the CSV settings by sampling from the provided file.

In [13]:
# read from a file using fully auto-detected settings
con.read_csv('cities.csv')

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │
│     6 │ Mariehamn        │ ALD     │    60.097 │    19.949 │      10682 │
│     7 │ Ramallah         │ PSE     │  31.90294 │  35.20621 │      24599 │
│     8 │ Vatican City     │ VAT     │  41.90001 │  12.44781 │        832 │
│     9 │ Poitier          │ FRA     │  46.58329 │   0.33328 │      85960 │
│    10 │ Cl

In [14]:
# specify options on how the CSV is formatted internally
con.read_csv('cities.csv', header=True, sep=',')

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │
│     6 │ Mariehamn        │ ALD     │    60.097 │    19.949 │      10682 │
│     7 │ Ramallah         │ PSE     │  31.90294 │  35.20621 │      24599 │
│     8 │ Vatican City     │ VAT     │  41.90001 │  12.44781 │        832 │
│     9 │ Poitier          │ FRA     │  46.58329 │   0.33328 │      85960 │
│    10 │ Cl

In [15]:
# use the (experimental) parallel CSV reader
con.read_csv('cities.csv', parallel=True)

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │
│     6 │ Mariehamn        │ ALD     │    60.097 │    19.949 │      10682 │
│     7 │ Ramallah         │ PSE     │  31.90294 │  35.20621 │      24599 │
│     8 │ Vatican City     │ VAT     │  41.90001 │  12.44781 │        832 │
│     9 │ Poitier          │ FRA     │  46.58329 │   0.33328 │      85960 │
│    10 │ Cl

In [16]:
# directly read a CSV file from within SQL
con.sql("SELECT * FROM 'cities.csv'")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │
│     6 │ Mariehamn        │ ALD     │    60.097 │    19.949 │      10682 │
│     7 │ Ramallah         │ PSE     │  31.90294 │  35.20621 │      24599 │
│     8 │ Vatican City     │ VAT     │  41.90001 │  12.44781 │        832 │
│     9 │ Poitier          │ FRA     │  46.58329 │   0.33328 │      85960 │
│    10 │ Cl

In [17]:
# call read_csv from within SQL
con.sql("SELECT * FROM read_csv_auto('cities.csv')")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │
│     6 │ Mariehamn        │ ALD     │    60.097 │    19.949 │      10682 │
│     7 │ Ramallah         │ PSE     │  31.90294 │  35.20621 │      24599 │
│     8 │ Vatican City     │ VAT     │  41.90001 │  12.44781 │        832 │
│     9 │ Poitier          │ FRA     │  46.58329 │   0.33328 │      85960 │
│    10 │ Cl

## JSON Files

JSON files can be read using the `read_json` function, called either from within Python or directly from within SQL. By default, the `read_json` function will automatically detect if a file contains newline-delimited JSON or regular JSON, and will detect the schema of the objects stored within the JSON file.

In [18]:
# read from a single JSON file
con.read_json('cities.json')

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │
│     6 │ Mariehamn        │ ALD     │    60.097 │    19.949 │      10682 │
│     7 │ Ramallah         │ PSE     │  31.90294 │  35.20621 │      24599 │
│     8 │ Vatican City     │ VAT     │  41.90001 │  12.44781 │        832 │
│     9 │ Poitier          │ FRA     │  46.58329 │   0.33328 │      85960 │
│    10 │ Cl

In [19]:
# directly read a JSON file from within SQL
con.sql("SELECT * FROM 'cities.json'")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │
│     6 │ Mariehamn        │ ALD     │    60.097 │    19.949 │      10682 │
│     7 │ Ramallah         │ PSE     │  31.90294 │  35.20621 │      24599 │
│     8 │ Vatican City     │ VAT     │  41.90001 │  12.44781 │        832 │
│     9 │ Poitier          │ FRA     │  46.58329 │   0.33328 │      85960 │
│    10 │ Cl

In [20]:
# call read_json from within SQL
con.sql("SELECT * FROM read_json_auto('cities.json')")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │
│     6 │ Mariehamn        │ ALD     │    60.097 │    19.949 │      10682 │
│     7 │ Ramallah         │ PSE     │  31.90294 │  35.20621 │      24599 │
│     8 │ Vatican City     │ VAT     │  41.90001 │  12.44781 │        832 │
│     9 │ Poitier          │ FRA     │  46.58329 │   0.33328 │      85960 │
│    10 │ Cl

## DataFrames

DuckDB is automatically able to query a Pandas DataFrame.

In [21]:
df = pd.read_csv('cities.csv')
df

,id,name,country,latitude,longitude,population
0,1,Bombo,UGA,0.58330,32.53330,75000
1,2,Fort Portal,UGA,0.67100,30.27500,42670
2,3,Potenza,ITA,40.64200,15.79900,69060
3,4,Campobasso,ITA,41.56300,14.65600,50762
4,5,Aosta,ITA,45.73700,7.31500,34062
...,...,...,...,...,...,...
1244,1245,Rio de Janeiro,BRA,-22.92502,-43.22502,11748000
1245,1246,Sao Paulo,BRA,-23.55868,-46.62502,18845000
1246,1247,Sydney,AUS,-33.92001,151.18518,4630000
1247,1248,Singapore,SGP,1.29303,103.85582,5183700


In [22]:
con.sql('SELECT * FROM df').fetchall()

[(1, 'Bombo', 'UGA', 0.5833, 32.5333, 75000),
 (2, 'Fort Portal', 'UGA', 0.671, 30.275, 42670),
 (3, 'Potenza', 'ITA', 40.642, 15.799, 69060),
 (4, 'Campobasso', 'ITA', 41.563, 14.656, 50762),
 (5, 'Aosta', 'ITA', 45.737, 7.315, 34062),
 (6, 'Mariehamn', 'ALD', 60.097, 19.949, 10682),
 (7, 'Ramallah', 'PSE', 31.90294, 35.20621, 24599),
 (8, 'Vatican City', 'VAT', 41.90001, 12.44781, 832),
 (9, 'Poitier', 'FRA', 46.58329, 0.33328, 85960),
 (10, 'Clermont-Ferrand', 'FRA', 45.77998, 3.08001, 233050),
 (11, 'Besancon', 'FRA', 47.23, 6.03001, 128426),
 (12, 'Artigas Base', 'ATA', -62.17389, -58.86386, 60),
 (13, 'Chipata', 'ZMB', -13.62957, 32.64001, 85963),
 (14, 'Jinja', 'UGA', 0.44042, 33.19993, 301619),
 (15, 'Arua', 'UGA', 3.02037, 30.90002, 250000),
 (16, 'Mbale', 'UGA', 1.09041, 34.17, 402368),
 (17, 'Moroto', 'UGA', 2.54035, 34.63999, 371),
 (18, 'Masaka', 'UGA', -0.32961, 31.72999, 65373),
 (19, 'Mbarara', 'UGA', -0.59962, 30.65, 83700),
 (20, 'Otjiwarongo', 'NAM', -20.45954, 16.64

## Parquet Files

Parquet files can be read using the `read_parquet` function, called either from within Python or directly from within SQL.

In [23]:
# read from a single Parquet file
con.read_parquet('cities.parquet')

┌─────────┬───────────────────────────────────────────┬────────┬───────────┬───────────┬──────────────────┬────────────┐
│ country │                 geometry                  │   id   │ latitude  │ longitude │       name       │ population │
│ varchar │                   blob                    │ double │  double   │  double   │     varchar      │   double   │
├─────────┼───────────────────────────────────────────┼────────┼───────────┼───────────┼──────────────────┼────────────┤
│ UGA     │ \x01\x01\x00\x00\x00xz\xA5,CD@@\xB57\xF…  │    1.0 │    0.5833 │   32.5333 │ Bombo            │    75000.0 │
│ UGA     │ \x01\x01\x00\x00\x00fffffF>@F\xB6\xF3\x…  │    2.0 │     0.671 │    30.275 │ Fort Portal      │    42670.0 │
│ ITA     │ \x01\x01\x00\x00\x00\x0C\x02+\x87\x16\x…  │    3.0 │    40.642 │    15.799 │ Potenza          │    69060.0 │
│ ITA     │ \x01\x01\x00\x00\x00\x1DZd;\xDFO-@\xF2\…  │    4.0 │    41.563 │    14.656 │ Campobasso       │    50762.0 │
│ ITA     │ \x01\x01\x00\x00\x00

In [24]:
# directly read a Parquet file from within SQL
con.sql("SELECT * FROM 'cities.parquet'")

┌─────────┬───────────────────────────────────────────┬────────┬───────────┬───────────┬──────────────────┬────────────┐
│ country │                 geometry                  │   id   │ latitude  │ longitude │       name       │ population │
│ varchar │                   blob                    │ double │  double   │  double   │     varchar      │   double   │
├─────────┼───────────────────────────────────────────┼────────┼───────────┼───────────┼──────────────────┼────────────┤
│ UGA     │ \x01\x01\x00\x00\x00xz\xA5,CD@@\xB57\xF…  │    1.0 │    0.5833 │   32.5333 │ Bombo            │    75000.0 │
│ UGA     │ \x01\x01\x00\x00\x00fffffF>@F\xB6\xF3\x…  │    2.0 │     0.671 │    30.275 │ Fort Portal      │    42670.0 │
│ ITA     │ \x01\x01\x00\x00\x00\x0C\x02+\x87\x16\x…  │    3.0 │    40.642 │    15.799 │ Potenza          │    69060.0 │
│ ITA     │ \x01\x01\x00\x00\x00\x1DZd;\xDFO-@\xF2\…  │    4.0 │    41.563 │    14.656 │ Campobasso       │    50762.0 │
│ ITA     │ \x01\x01\x00\x00\x00

In [25]:
# call read_parquet from within SQL
con.sql("SELECT * FROM read_parquet('cities.parquet')")

┌─────────┬───────────────────────────────────────────┬────────┬───────────┬───────────┬──────────────────┬────────────┐
│ country │                 geometry                  │   id   │ latitude  │ longitude │       name       │ population │
│ varchar │                   blob                    │ double │  double   │  double   │     varchar      │   double   │
├─────────┼───────────────────────────────────────────┼────────┼───────────┼───────────┼──────────────────┼────────────┤
│ UGA     │ \x01\x01\x00\x00\x00xz\xA5,CD@@\xB57\xF…  │    1.0 │    0.5833 │   32.5333 │ Bombo            │    75000.0 │
│ UGA     │ \x01\x01\x00\x00\x00fffffF>@F\xB6\xF3\x…  │    2.0 │     0.671 │    30.275 │ Fort Portal      │    42670.0 │
│ ITA     │ \x01\x01\x00\x00\x00\x0C\x02+\x87\x16\x…  │    3.0 │    40.642 │    15.799 │ Potenza          │    69060.0 │
│ ITA     │ \x01\x01\x00\x00\x00\x1DZd;\xDFO-@\xF2\…  │    4.0 │    41.563 │    14.656 │ Campobasso       │    50762.0 │
│ ITA     │ \x01\x01\x00\x00\x00

## GeoJSON Files

In [26]:
con.sql('SELECT * FROM ST_Drivers()')

┌────────────────┬──────────────────────┬────────────┬──────────┬──────────┬───────────────────────────────────────────┐
│   short_name   │      long_name       │ can_create │ can_copy │ can_open │                 help_url                  │
│    varchar     │       varchar        │  boolean   │ boolean  │ boolean  │                  varchar                  │
├────────────────┼──────────────────────┼────────────┼──────────┼──────────┼───────────────────────────────────────────┤
│ ESRI Shapefile │ ESRI Shapefile       │ true       │ false    │ true     │ https://gdal.org/drivers/vector/shapefi…  │
│ MapInfo File   │ MapInfo File         │ true       │ false    │ true     │ https://gdal.org/drivers/vector/mitab.h…  │
│ UK .NTF        │ UK .NTF              │ false      │ false    │ true     │ https://gdal.org/drivers/vector/ntf.html  │
│ LVBAG          │ Kadaster LV BAG Ex…  │ false      │ false    │ true     │ https://gdal.org/drivers/vector/lvbag.h…  │
│ S57            │ IHO S-57 (ENC

In [27]:
con.sql("SELECT * FROM ST_Read('cities.geojson')")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┬─────────────────────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │            geom             │
│ int32 │     varchar      │ varchar │  double   │  double   │   int32    │          geometry           │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┼─────────────────────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │ POINT (32.5333 0.5833)      │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │ POINT (30.275 0.671)        │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │ POINT (15.799 40.642)       │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │ POINT (14.656 41.563)       │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │ POINT (7.315 45.737)        │
│     6 │ Mariehamn        │ ALD     │    60.0

In [28]:
con.sql("FROM ST_Read('cities.geojson')")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┬─────────────────────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │            geom             │
│ int32 │     varchar      │ varchar │  double   │  double   │   int32    │          geometry           │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┼─────────────────────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │ POINT (32.5333 0.5833)      │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │ POINT (30.275 0.671)        │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │ POINT (15.799 40.642)       │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │ POINT (14.656 41.563)       │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │ POINT (7.315 45.737)        │
│     6 │ Mariehamn        │ ALD     │    60.0

In [29]:
con.sql("CREATE TABLE cities AS SELECT * FROM ST_Read('cities.geojson')")

In [30]:
con.table('cities')

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┬─────────────────────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │            geom             │
│ int32 │     varchar      │ varchar │  double   │  double   │   int32    │          geometry           │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┼─────────────────────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │ POINT (32.5333 0.5833)      │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │ POINT (30.275 0.671)        │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │ POINT (15.799 40.642)       │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │ POINT (14.656 41.563)       │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │ POINT (7.315 45.737)        │
│     6 │ Mariehamn        │ ALD     │    60.0

In [31]:
con.sql("SELECT * FROM cities")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┬─────────────────────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │            geom             │
│ int32 │     varchar      │ varchar │  double   │  double   │   int32    │          geometry           │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┼─────────────────────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │ POINT (32.5333 0.5833)      │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │ POINT (30.275 0.671)        │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │ POINT (15.799 40.642)       │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │ POINT (14.656 41.563)       │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │ POINT (7.315 45.737)        │
│     6 │ Mariehamn        │ ALD     │    60.0

## Shapefiles

In [32]:
con.sql("SELECT * FROM ST_Read('cities.shp')")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┬─────────────────────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │            geom             │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │          geometry           │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┼─────────────────────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │ POINT (32.5333 0.5833)      │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │ POINT (30.275 0.671)        │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │ POINT (15.799 40.642)       │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │ POINT (14.656 41.563)       │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │ POINT (7.315 45.737)        │
│     6 │ Mariehamn        │ ALD     │    60.0

In [33]:
con.sql("FROM ST_Read('cities.shp')")

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┬─────────────────────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │            geom             │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │          geometry           │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┼─────────────────────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │ POINT (32.5333 0.5833)      │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │ POINT (30.275 0.671)        │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │ POINT (15.799 40.642)       │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │ POINT (14.656 41.563)       │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │ POINT (7.315 45.737)        │
│     6 │ Mariehamn        │ ALD     │    60.0

In [34]:
con.sql(
    """
        CREATE TABLE IF NOT EXISTS cities2 AS 
        SELECT * FROM ST_Read('cities.shp')
        """
)

In [35]:
con.table('cities2')

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┬─────────────────────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │            geom             │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │          geometry           │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┼─────────────────────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │ POINT (32.5333 0.5833)      │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │ POINT (30.275 0.671)        │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │ POINT (15.799 40.642)       │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │ POINT (14.656 41.563)       │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │ POINT (7.315 45.737)        │
│     6 │ Mariehamn        │ ALD     │    60.0

In [36]:
con.sql('SELECT * FROM cities2')

┌───────┬──────────────────┬─────────┬───────────┬───────────┬────────────┬─────────────────────────────┐
│  id   │       name       │ country │ latitude  │ longitude │ population │            geom             │
│ int64 │     varchar      │ varchar │  double   │  double   │   int64    │          geometry           │
├───────┼──────────────────┼─────────┼───────────┼───────────┼────────────┼─────────────────────────────┤
│     1 │ Bombo            │ UGA     │    0.5833 │   32.5333 │      75000 │ POINT (32.5333 0.5833)      │
│     2 │ Fort Portal      │ UGA     │     0.671 │    30.275 │      42670 │ POINT (30.275 0.671)        │
│     3 │ Potenza          │ ITA     │    40.642 │    15.799 │      69060 │ POINT (15.799 40.642)       │
│     4 │ Campobasso       │ ITA     │    41.563 │    14.656 │      50762 │ POINT (14.656 41.563)       │
│     5 │ Aosta            │ ITA     │    45.737 │     7.315 │      34062 │ POINT (7.315 45.737)        │
│     6 │ Mariehamn        │ ALD     │    60.0

## GeoParquet Files

In [37]:
con.sql("SELECT * FROM 'cities.parquet'")

┌─────────┬───────────────────────────────────────────┬────────┬───────────┬───────────┬──────────────────┬────────────┐
│ country │                 geometry                  │   id   │ latitude  │ longitude │       name       │ population │
│ varchar │                   blob                    │ double │  double   │  double   │     varchar      │   double   │
├─────────┼───────────────────────────────────────────┼────────┼───────────┼───────────┼──────────────────┼────────────┤
│ UGA     │ \x01\x01\x00\x00\x00xz\xA5,CD@@\xB57\xF…  │    1.0 │    0.5833 │   32.5333 │ Bombo            │    75000.0 │
│ UGA     │ \x01\x01\x00\x00\x00fffffF>@F\xB6\xF3\x…  │    2.0 │     0.671 │    30.275 │ Fort Portal      │    42670.0 │
│ ITA     │ \x01\x01\x00\x00\x00\x0C\x02+\x87\x16\x…  │    3.0 │    40.642 │    15.799 │ Potenza          │    69060.0 │
│ ITA     │ \x01\x01\x00\x00\x00\x1DZd;\xDFO-@\xF2\…  │    4.0 │    41.563 │    14.656 │ Campobasso       │    50762.0 │
│ ITA     │ \x01\x01\x00\x00\x00

In [38]:
con.sql(
    """
CREATE TABLE IF NOT EXISTS cities3 AS
SELECT * EXCLUDE geometry, ST_GeomFromWKB(geometry) 
AS geometry FROM 'cities.parquet'
"""
)

In [39]:
con.table('cities3')

┌─────────┬────────┬───────────┬───────────┬──────────────────┬────────────┬─────────────────────────────┐
│ country │   id   │ latitude  │ longitude │       name       │ population │          geometry           │
│ varchar │ double │  double   │  double   │     varchar      │   double   │          geometry           │
├─────────┼────────┼───────────┼───────────┼──────────────────┼────────────┼─────────────────────────────┤
│ UGA     │    1.0 │    0.5833 │   32.5333 │ Bombo            │    75000.0 │ POINT (32.5333 0.5833)      │
│ UGA     │    2.0 │     0.671 │    30.275 │ Fort Portal      │    42670.0 │ POINT (30.275 0.671)        │
│ ITA     │    3.0 │    40.642 │    15.799 │ Potenza          │    69060.0 │ POINT (15.799 40.642)       │
│ ITA     │    4.0 │    41.563 │    14.656 │ Campobasso       │    50762.0 │ POINT (14.656 41.563)       │
│ ITA     │    5.0 │    45.737 │     7.315 │ Aosta            │    34062.0 │ POINT (7.315 45.737)        │
│ ALD     │    6.0 │    60.097 │    1

In [40]:
con.sql(
    """
CREATE TABLE IF NOT EXISTS country AS
SELECT * EXCLUDE geometry, ST_GeomFromWKB(geometry) FROM
        's3://us-west-2.opendata.source.coop/google-research-open-buildings/v2/geoparquet-admin1/country=SSD/*.parquet'
"""
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [41]:
con.table('country')

┌─────────┬───────────────────┬────────────────┬────────────┬────────────────┬─────────────────────────────────────────┐
│ country │      admin_1      │ area_in_meters │ confidence │ full_plus_code │        st_geomfromwkb(geometry)         │
│ varchar │      varchar      │     double     │   double   │    varchar     │                geometry                 │
├─────────┼───────────────────┼────────────────┼────────────┼────────────────┼─────────────────────────────────────────┤
│ SSD     │ Central Equatoria │        13.2942 │     0.6061 │ 6GMGFRQX+797F  │ POLYGON ((30.8484392928101 3.48814927…  │
│ SSD     │ Central Equatoria │        17.9561 │     0.7432 │ 6GMGFRQX+7999  │ POLYGON ((30.8485154080012 3.48817201…  │
│ SSD     │ Central Equatoria │        16.1245 │     0.7016 │ 6GMGFRQX+WXHQ  │ POLYGON ((30.8500129796786 3.48979698…  │
│ SSD     │ Central Equatoria │        20.5568 │     0.7655 │ 6GMGFRQX+XX66  │ POLYGON ((30.849898767126 3.489886617…  │
│ SSD     │ Central Equatoria │ 

In [42]:
con.sql('SELECT COUNT(*) FROM country')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        37781 │
└──────────────┘

## References

- [DuckDB Data Ingestion](https://duckdb.org/docs/api/python/data_ingestion)